In [0]:
!nvidia-smi

Tue Sep 17 07:47:53 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!pip install -q keras
!pip install -q pandas
!pip install -q numpy
!pip install -q tensorflow
!pip install -U -q PyDrive

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("drive/My Drive/colab_datas/teknofest/keras-retinanet")

In [0]:
!pip install -q keras_resnet

In [0]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
import glob
import pandas as pd
# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf
from tqdm import tqdm

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


## Video Pred

In [0]:
input_video="drone_1"

vidcap = cv2.VideoCapture('drone_night_footage/{}.mp4'.format(input_video))
success,image = vidcap.read()
count = 0
frames=[]
while success:
    frames.append(image)     # save frame as JPEG file      
    success,image = vidcap.read()
    # print('Read a new frame: ', success)
    count += 1

In [0]:
frames=glob.glob("photos/test_images/*")

In [0]:
labels_to_names = {0: 'yaya', 1: 'arac'}
!python keras_retinanet/bin/convert_model.py ./snapshots_woconfig_waug/resnet50_csv_11.h5 ./snapshots/infer_model.h5
model = models.load_model("./snapshots/infer_model.h5", backbone_name='resnet50')
resnet50_images=[]
for image in tqdm(frames):# load image
    sha=image.shape
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    image = preprocess_image(image)
    image, scale = resize_image(image,max_side=3000)
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))

    boxes /= scale

    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < 0.5:
            break
        color = label_color(label)
        b = box.astype(int)
        draw_box(draw, b, color=color)
        
        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
    resnet50_images.append(draw)
# os.mkdir("snapshots_resnet50_csv_01")
# for i in range(len(resnet50_images)):
#     image_=cv2.cvtColor(resnet50_images[i],cv2.COLOR_BGR2RGB)
#     cv2.imwrite("snapshots_resnet50_csv_01/{}.jpg".format(i),image_)
#     # plt.figure(figsize=(20, 20))
#     # plt.axis('off')
#     # plt.imshow(image)
#     # plt.show()
#     # plt.pause(0.01)

Using TensorFlow backend.


2019-09-16 07:05:15.159799: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-09-16 07:05:15.160130: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x252f640 executing computations on platform Host. Devices:
2019-09-16 07:05:15.160164: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2019-09-16 07:05:15.161986: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-09-16 07:05:15.164219: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2019-09-16 07:05:15.164281: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: 3da8a168a621
2019-09-16 07:05:15.164309: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: 3da8a168a621
2019-09-16 07

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
100%|██████████| 721/721 [04:12<00:00,  2.84it/s]


In [0]:
from os.path import isfile, join
height, width, layers = resnet50_images[0].shape

out = cv2.VideoWriter("drone_night_footage/{}_pred_lastest_waug.avi".format(input_video),cv2.VideoWriter_fourcc(*"MJPG"), 60, (width,height))
for i in tqdm(range(len(resnet50_images))):
    out.write(resnet50_images[i])
    # cv2.imwrite("drone_night_footage/"+str(i)+".jpg",resnet50_images[i])
cv2.destroyAllWindows()
out.release()

100%|██████████| 721/721 [00:02<00:00, 244.98it/s]


In [0]:
# [os.remove(i) for i in glob.glob("drone_night_footage/*.jpg")]

## Modellerin birleşmesi

In [0]:
paths=glob.glob("photos/test_images/*")

In [0]:
# 4 farklı modelin karsılastırılması 

model_paths=["./snapshots/son_50.h5","./snapshots/resnet101_csv_06.h5","./snapshots_woconfig/resnet50_csv_14.h5","./snapshots_woconfig_waug/resnet50_csv_11.h5"]
labels_to_names = {0: 'yaya', 1: 'arac'}
pred_dict={}
for model_path in model_paths:
    !python keras_retinanet/bin/convert_model.py $model_path ./snapshots/infer_model.h5
    if "resnet101" in model_path:
        model = models.load_model("./snapshots/infer_model.h5", backbone_name='resnet101')
    else:
        model = models.load_model("./snapshots/infer_model.h5", backbone_name='resnet50')
    pred_dict[model_path]=[]
    print(model_path)
    for path in tqdm(paths):
        image=read_image_bgr(path)
        sha=image.shape
        # draw = image.copy()
        # draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
        image = preprocess_image(image)
        image, scale = resize_image(image,max_side=3000)
        boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
        boxes /= scale
        bbs=[]
        for box, score, label in zip(boxes[0], scores[0], labels[0]):
            if score < 0.5:
                break
            # color = label_color(label)
            b = box.astype(int)
            bbs.append([path,b[0],b[1],b[2],b[3],labels_to_names[label], score])
            # draw_box(draw, b, color=color)
            # caption = "{} {:.3f}".format(labels_to_names[label], score)
            # draw_caption(draw, b, caption)
        # font = cv2.FONT_HERSHEY_SIMPLEX
        # cv2.putText(draw,model_path.split("/")[-1].split(".")[0],(10,500), font, 4,(255,255,255),2,cv2.LINE_AA)
        pred_dict[model_path].append(bbs)

In [0]:
pred_dict.keys()

dict_keys(['./snapshots/son_50.h5', './snapshots/resnet101_csv_06.h5', './snapshots_woconfig/resnet50_csv_14.h5', './snapshots_woconfig_waug/resnet50_csv_11.h5'])

In [0]:
# pred_dict["./snapshots/son_50.h5"]=[i for i in pred_dict["./snapshots/son_50.h5"]  if not len(i)==0]
# pred_dict["./snapshots/resnet101_csv_06.h5"]=[i for i in pred_dict["./snapshots/resnet101_csv_06.h5"]  if not len(i)==0]
# pred_dict["./snapshots_woconfig_waug/resnet50_csv_11.h5"]=[i for i in pred_dict["./snapshots_woconfig_waug/resnet50_csv_11.h5"]  if not len(i)==0]
# pred_dict["./snapshots_woconfig/resnet50_csv_14.h5"]=[i for i in pred_dict["./snapshots_woconfig/resnet50_csv_14.h5"]  if not len(i)==0]

In [0]:
df=[]
for key in pred_dict.keys():
  for frame in pred_dict[key]:
    for pred in frame[:-1]:
      pred.append(key)
      df.append(pred)

In [0]:
df=pd.DataFrame(df)
df

In [0]:
df.to_csv("pred_model_comparison.csv",index=False)

In [0]:
df.head()

,0,1,2,3,4,5,6,7
0,photos/test_images/3350.jpg,1527,281,1569,318,arac,0.992526,./snapshots/son_50.h5
1,photos/test_images/3350.jpg,805,82,839,111,arac,0.989626,./snapshots/son_50.h5
2,photos/test_images/3350.jpg,1479,49,1525,107,arac,0.966519,./snapshots/son_50.h5
3,photos/test_images/3350.jpg,1506,496,1547,568,arac,0.836250,./snapshots/son_50.h5
4,photos/test_images/3351.jpg,1493,74,1540,116,arac,0.974509,./snapshots/son_50.h5


In [0]:
df[7].value_counts().index

Index(['./snapshots/son_50.h5', './snapshots_woconfig_waug/resnet50_csv_11.h5',
       './snapshots_woconfig/resnet50_csv_14.h5',
       './snapshots/resnet101_csv_06.h5'],
      dtype='object')

In [0]:
labels_to_names_rev={'yaya':0,  'arac':1}

In [0]:
frame_paths=pd.DataFrame(df[0].value_counts().index)
frame_paths[1]=frame_paths[0].apply(lambda x:x.split("/")[-1].split(".")[0]).astype(int)
frame_paths=frame_paths.sort_values(by=[1])
frame_paths

In [0]:
from os.path import isfile, join
# height, width, layers = image.shape

out = cv2.VideoWriter("project_11.avi",cv2.VideoWriter_fourcc(*"DIVX"), 7.5, (1920,1080))

for frame in tqdm(frame_paths[0].values):
    image=cv2.imread(frame)
    images_dict={}
    models=df[7].value_counts().index
    for model_path in models:
        image_1=image.copy()
        unq_df=df.loc[(df[7]==model_path)&(df[0]==frame)]
        for bb in unq_df.values:
            color = label_color(labels_to_names_rev[bb[5]])
            draw_box(image_1, bb[1:5], color=color)
            caption = "{} {:.3f}".format(bb[5], bb[6])
            draw_caption(image_1, bb[1:5], caption)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(image_1,model_path.split("/")[-1].split(".")[0],(10,500), font, 4,(255,255,255),2,cv2.LINE_AA)
        images_dict[model_path]=image_1.copy()
    image_1=np.concatenate((images_dict["./snapshots/son_50.h5"],images_dict["./snapshots_woconfig_waug/resnet50_csv_11.h5"]),axis=0)
    image_2=np.concatenate((images_dict["./snapshots_woconfig/resnet50_csv_14.h5"],images_dict["./snapshots/resnet101_csv_06.h5"]),axis=0)
    image_1=np.concatenate((image_1,image_2),axis=1)
    image_1=cv2.resize(image_1,(1920,1080))
    out.write(image_1)
cv2.destroyAllWindows()
out.release()

## Video cıkar



In [0]:
paths=glob.glob("photos/test_images/*")


In [100]:
from os.path import isfile, join
# height, width, layers = image.shape
out = cv2.VideoWriter("project_1222.avi",cv2.VideoWriter_fourcc(*"DIVX"), 7.5, (960,540))

for frame in tqdm(paths[2000:4000]):
    image=cv2.imread(frame)
    images_dict={}
    image_1=image.copy()
    image_1=cv2.cvtColor(image_1,cv2.COLOR_BGR2RGB)
    unq_df=model_preds[model_preds[0]==frame]
    for bb in unq_df.values:
        color = label_color(labels_to_names_rev[bb[5]])
        draw_box(image_1, bb[1:5], color=color)
        caption = "{} {:.3f}".format(bb[5], bb[6])
        draw_caption(image_1, bb[1:5], caption)
    image_1=cv2.resize(image_1,(960,540))
    out.write(image_1)
cv2.destroyAllWindows()
out.release()



  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 1/2000 [00:00<13:27,  2.47it/s]

  0%|          | 2/2000 [00:00<13:15,  2.51it/s]

  0%|          | 3/2000 [00:01<11:46,  2.83it/s]

  0%|          | 4/2000 [00:01<10:45,  3.09it/s]

  0%|          | 5/2000 [00:01<10:07,  3.29it/s]

  0%|          | 6/2000 [00:01<10:48,  3.08it/s]

  0%|          | 7/2000 [00:02<11:09,  2.97it/s]

  0%|          | 8/2000 [00:02<14:52,  2.23it/s]

  0%|          | 9/2000 [00:03<14:01,  2.37it/s]

  0%|          | 10/2000 [00:03<13:43,  2.42it/s]

  1%|          | 11/2000 [00:04<14:35,  2.27it/s]

  1%|          | 12/2000 [00:04<16:15,  2.04it/s]

  1%|          | 13/2000 [00:05<15:24,  2.15it/s]

  1%|          | 14/2000 [00:05<15:29,  2.14it/s]

  1%|          | 15/2000 [00:06<16:56,  1.95it/s]

  1%|          | 16/2000 [00:06<15:58,  2.07it/s]

  1%|          | 17/2000 [00:07<14:56,  2.21it/s]

  1%|          | 18/2000 [00:07<14:55,  2.21it/s]

  1%|          | 19/2000 [00:08<14:38,  2.25it/

## Blended predictions

In [0]:
preds=pd.read_csv("./pred_model_comparison.csv")
preds.columns=["path","x1","y1","x2","y2","label","confidence","model"]
preds["area"]=(preds["x2"]-preds["x1"])*(preds["y2"]-preds["y1"])
preds["center_x"]=((preds["x2"].astype(int)+preds["x1"].astype(int))/2).astype(int)
preds["center_y"]=((preds["y2"].astype(int)+preds["y1"].astype(int))/2).astype(int)
preds=preds.sort_values(by=["path","center_x","center_y"])
preds.head()

,path,x1,y1,x2,y2,label,confidence,model,area,center_x,center_y
29716,photos/test_images/100.jpg,683,234,740,296,arac,0.974719,./snapshots/resnet101_csv_06.h5,3534,711,265
15062,photos/test_images/100.jpg,686,236,739,297,arac,0.997402,./snapshots/son_50.h5,3233,712,266
44417,photos/test_images/100.jpg,687,236,740,297,arac,0.999925,./snapshots_woconfig/resnet50_csv_14.h5,3233,713,266
59456,photos/test_images/100.jpg,685,235,742,298,arac,0.921941,./snapshots_woconfig_waug/resnet50_csv_11.h5,3591,713,266
15063,photos/test_images/100.jpg,1859,441,1908,482,yaya,0.509156,./snapshots/son_50.h5,2009,1883,461


In [0]:
preds=preds.drop(preds.loc[preds["model"]=="./snapshots/resnet101_csv_06.h5"].index)
preds

,path,x1,y1,x2,y2,label,confidence,model,area,center_x,center_y
15062,photos/test_images/100.jpg,686,236,739,297,arac,0.997402,./snapshots/son_50.h5,3233,712,266
44417,photos/test_images/100.jpg,687,236,740,297,arac,0.999925,./snapshots_woconfig/resnet50_csv_14.h5,3233,713,266
59456,photos/test_images/100.jpg,685,235,742,298,arac,0.921941,./snapshots_woconfig_waug/resnet50_csv_11.h5,3591,713,266
15063,photos/test_images/100.jpg,1859,441,1908,482,yaya,0.509156,./snapshots/son_50.h5,2009,1883,461
43933,photos/test_images/1000.jpg,41,278,106,312,arac,0.899445,./snapshots_woconfig/resnet50_csv_14.h5,2210,73,295
58942,photos/test_images/1000.jpg,43,285,107,311,arac,0.703021,./snapshots_woconfig_waug/resnet50_csv_11.h5,1664,75,298
14518,photos/test_images/1000.jpg,272,393,345,436,arac,0.999637,./snapshots/son_50.h5,3139,308,414
43929,photos/test_images/1000.jpg,272,394,345,437,arac,0.999989,./snapshots_woconfig/resnet50_csv_14.h5,3139,308,415
58936,photos/test_images/1000.jpg,272,394,346,439,arac,0.961082,./snapshots_woconfig_waug/resnet50_csv_11.h5,3330,309,416
14522,photos/test_images/1000.jpg,404,357,474,400,arac,0.994914,./snapshots/son_50.h5,3010,439,378


In [0]:
frame_paths=pd.DataFrame(preds["path"].value_counts().index)
frame_paths[1]=frame_paths[0].apply(lambda x:x.split("/")[-1].split(".")[0]).astype(int)
frame_paths=frame_paths.sort_values(by=[1])
frame_paths.head()

,0,1
3591,photos/test_images/5.jpg,5
3607,photos/test_images/7.jpg,7
3570,photos/test_images/8.jpg,8
3626,photos/test_images/9.jpg,9
3567,photos/test_images/11.jpg,11


In [0]:
for i, row in preds.iterrows():
    break
i,row

(15062, path          photos/test_images/100.jpg
 x1                                   686
 y1                                   236
 x2                                   739
 y2                                   297
 label                               arac
 confidence                      0.997402
 model              ./snapshots/son_50.h5
 area                                3233
 center_x                             712
 center_y                             266
 Name: 15062, dtype: object)

In [0]:
# buraya iki data data frame i alarak ensemble i koy

def find_intersection(df1,df2):
    new_preds=[]
    idx_tbd_1=[]
    idx_tbd_2=[]
    for idx_df1,bb_df1 in df1.iterrows():
        for idx_df2,bb_df2 in df2.iterrows():
            if abs(bb_df1["center_x"]-bb_df2["center_x"])<=20 and abs(bb_df1["center_y"]-bb_df2["center_y"])<=20:
                frame=bb_df1["path"]
                x1=min(bb_df1["x1"],bb_df2["x1"])
                y1=min(bb_df1["y1"],bb_df2["y1"])
                x2=min(bb_df1["x2"],bb_df2["x2"])
                y2=min(bb_df1["y2"],bb_df2["y2"])
                label_=bb_df1["label"]
                conf=bb_df1["confidence"]
                idx_tbd_1.append(idx_df1)
                idx_tbd_2.append(idx_df2)
                new_preds.append([frame,x1,y1,x2,y2,label_,conf,"ens",(x2-x1)*(y2-y1),bb_df1["center_x"],bb_df1["center_y"]])
    df1=df1.drop(idx_tbd_1)
    df2=df2.drop(idx_tbd_2)
    new_preds=pd.DataFrame(new_preds)
    try:
        new_preds.columns=df1.columns
    except:
        pass
    return df1,df2,new_preds

In [95]:
indexes_tbd=[]
model_preds=[]
for frame in tqdm(frame_paths[0].values):
    unq_df=preds.loc[preds["path"]==frame]
    son_50_model=unq_df.loc[unq_df["model"]=="./snapshots/son_50.h5"]
    confident_model=unq_df.loc[unq_df["model"]=="./snapshots_woconfig/resnet50_csv_14.h5"]
    augmented_model=unq_df.loc[unq_df["model"]=="./snapshots_woconfig_waug/resnet50_csv_11.h5"]
    son_1,conf_1,ens_1=find_intersection(son_50_model,confident_model)
    ens_2,aug_1,ens_3=find_intersection(ens_1,augmented_model)
    for i in son_1.values:
        if not len(i)==0:
            model_preds.append(i)
    for i in conf_1.values:
        if not len(i)==0:
            model_preds.append(i)
    for i in ens_2.values:
        if not len(i)==0:
            model_preds.append(i)
    for i in aug_1.values:
        if not len(i)==0:
            model_preds.append(i)
    for i in ens_3.values:
        if not len(i)==0:
            model_preds.append(i)



  0%|          | 0/3634 [00:00<?, ?it/s]

  0%|          | 9/3634 [00:00<00:43, 82.59it/s]

  0%|          | 16/3634 [00:00<00:47, 76.15it/s]

  1%|          | 23/3634 [00:00<00:49, 72.71it/s]

  1%|          | 30/3634 [00:00<00:51, 70.16it/s]

  1%|          | 37/3634 [00:00<00:52, 68.79it/s]

  1%|          | 45/3634 [00:00<00:51, 69.58it/s]

  1%|▏         | 52/3634 [00:00<00:52, 68.24it/s]

  2%|▏         | 59/3634 [00:00<00:54, 66.17it/s]

  2%|▏         | 66/3634 [00:00<00:53, 66.40it/s]

  2%|▏         | 73/3634 [00:01<00:54, 65.00it/s]

  2%|▏         | 80/3634 [00:01<00:54, 65.20it/s]

  2%|▏         | 87/3634 [00:01<00:54, 65.09it/s]

  3%|▎         | 95/3634 [00:01<00:52, 67.23it/s]

  3%|▎         | 102/3634 [00:01<00:52, 67.31it/s]

  3%|▎         | 109/3634 [00:01<00:52, 67.31it/s]

  3%|▎         | 116/3634 [00:01<00:52, 66.73it/s]

  3%|▎         | 123/3634 [00:01<00:52, 67.23it/s]

  4%|▎         | 130/3634 [00:01<00:52, 66.55it/s]

  4%|▍         | 137/3634 [00:02<0

In [96]:
model_preds

[array(['photos/test_images/5.jpg', 1141, 530, 1229, 577, 'arac',
        0.5244427323341371, './snapshots_woconfig_waug/resnet50_csv_11.h5',
        4136, 1185, 553], dtype=object),
 array(['photos/test_images/7.jpg', 1137, 548, 1224, 599, 'arac',
        0.7210214138031006, './snapshots_woconfig_waug/resnet50_csv_11.h5',
        4437, 1180, 573], dtype=object),
 array(['photos/test_images/8.jpg', 759, 247, 843, 297, 'arac',
        0.5018519163131714, './snapshots_woconfig_waug/resnet50_csv_11.h5',
        4200, 801, 272], dtype=object),
 array(['photos/test_images/9.jpg', 1121, 561, 1212, 618, 'arac',
        0.6622017621994019, './snapshots_woconfig_waug/resnet50_csv_11.h5',
        5187, 1166, 589], dtype=object),
 array(['photos/test_images/11.jpg', 1097, 567, 1185, 623, 'arac',
        0.6417880058288574, './snapshots_woconfig_waug/resnet50_csv_11.h5',
        4928, 1141, 595], dtype=object),
 array(['photos/test_images/15.jpg', 1059, 578, 1135, 640, 'arac',
        0.9616463184

In [0]:
model_preds=pd.DataFrame(model_preds)

In [0]:
model_preds.to_csv("preds_tuned.csv",index=False,header=None)